In [2]:
import torch

In [3]:
batch_size = 4
num_mark = 2
seq_len = 5

In [4]:
last_event_time = torch.zeros(
            (batch_size, num_mark), dtype=torch.float32
        )
last_event_time

tensor([[0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.]])

In [5]:
event_seq = torch.randint(
            0, 2, (batch_size, seq_len), dtype=torch.float32
        )
event_seq

tensor([[1., 1., 0., 0., 0.],
        [0., 1., 1., 0., 1.],
        [1., 0., 1., 0., 1.],
        [0., 1., 1., 1., 1.]])

In [6]:
time_seq = torch.randn(
            (batch_size, seq_len), dtype=torch.float32
        ).abs()
time_seq = time_seq.cumsum(dim=1)
time_seq

tensor([[1.4714, 3.1033, 3.2334, 5.0800, 5.5261],
        [0.0943, 1.5425, 2.3193, 2.4992, 2.7180],
        [0.7317, 0.8511, 1.4375, 1.8325, 3.5591],
        [0.1244, 0.5622, 1.8200, 2.2945, 3.5384]])

In [7]:
for mark in range(num_mark):
            mark_mask = (event_seq == mark)  # [batch_size, seq_len]
            if mark_mask.any():
                # Opération vectorisée avec masquage efficace
                masked_times = time_seq.masked_fill(~mark_mask, float("-inf"))
                max_times, _ = masked_times.max(dim=1)
                valid_mask = (max_times != float("-inf"))
                last_event_time[valid_mask, mark] = max_times[valid_mask]
last_event_time

tensor([[5.5261, 3.1033],
        [2.4992, 2.7180],
        [1.8325, 3.5591],
        [0.1244, 3.5384]])

Le masque booléen joue le meme role que index_select en selectionnant seulement certaines colonnes/lignes ... dans une dimension

In [39]:
valid_mask = (last_event_time[:, 1]  > 3)
print("valid_mask: ", valid_mask)
type_pred = torch.randint(0, 2, (batch_size,))
print("type_pred:", type_pred, "shape: ", type_pred.shape)
last_event_time[valid_mask, type_pred]

valid_mask:  tensor([ True, False,  True,  True])
type_pred: tensor([0, 0, 1]) shape:  torch.Size([3])


tensor([5.5261, 1.8325, 3.5384])

# Exemples d'indexation avancée avec PyTorch

Ce notebook présente différentes techniques d'indexation avancée pour manipuler efficacement les tenseurs PyTorch.

## 1. Indexation de base et slicing

In [9]:
# Créons un tenseur 3D pour nos exemples
data = torch.randn(3, 4, 5)
print("Tenseur original shape:", data.shape)
print("Données:\n", data)

# Indexation basique
print("\n1. Premier élément (batch 0):", data[0].shape)
print("2. Dernière colonne:", data[:, :, -1].shape)
print("3. Slice du milieu:", data[:, 1:3, :].shape)

Tenseur original shape: torch.Size([3, 4, 5])
Données:
 tensor([[[ 2.1499,  0.6631,  0.1087, -0.4316,  1.3128],
         [ 0.6359,  1.3980,  0.8533,  0.3231, -0.8725],
         [ 1.3129,  0.8505, -0.2898,  0.9754,  0.8462],
         [-1.1346,  1.1589, -1.8925, -0.6487, -1.6561]],

        [[ 0.9511, -1.1479,  0.5666, -1.6383, -1.5595],
         [ 0.5956, -0.0333,  0.2996, -0.4183,  1.6169],
         [ 0.3581, -1.0088, -1.7175,  0.5518,  1.6188],
         [ 1.4274,  2.2002,  0.2962,  0.0888, -0.5887]],

        [[ 0.6040, -0.0827, -0.2848, -0.1057, -0.6897],
         [ 0.0200, -0.1890,  0.3926,  0.3901, -0.4661],
         [ 0.6775, -0.2156,  0.9627,  0.0132,  0.1663],
         [-1.5046,  1.9544,  1.2686, -1.6510, -1.8305]]])

1. Premier élément (batch 0): torch.Size([4, 5])
2. Dernière colonne: torch.Size([3, 4])
3. Slice du milieu: torch.Size([3, 2, 5])


## 2. Indexation avancée avec torch.gather

In [10]:
# torch.gather permet de sélectionner des éléments selon des indices
source = torch.tensor([[1, 2, 3, 4], 
                       [5, 6, 7, 8], 
                       [9, 10, 11, 12]])

# Indices à sélectionner pour chaque ligne
indices = torch.tensor([[0, 2], [1, 3], [0, 3]])

print("Source tensor:\n", source)
print("Indices:\n", indices)

# Gather sur la dimension 1 (colonnes)
gathered = torch.gather(source, dim=1, index=indices)
print("Résultat gather:\n", gathered)

# Exemple avec nos données de simulation : sélectionner le dernier événement de chaque type
batch_size, seq_len = 3, 5
events = torch.randint(0, 2, (batch_size, seq_len))
times = torch.randn(batch_size, seq_len).abs().cumsum(dim=1)

print("\nExemple avec événements:")
print("Events:\n", events)
print("Times:\n", times)

# Trouver l'index du dernier événement de type 1 pour chaque batch
last_indices = []
for b in range(batch_size):
    mask = events[b] == 1
    if mask.any():
        last_idx = torch.where(mask)[0][-1]
    else:
        last_idx = torch.tensor(0)  # fallback
    last_indices.append(last_idx)

last_indices = torch.stack(last_indices).unsqueeze(1)
print("Derniers indices pour type 1:\n", last_indices)

# Utiliser gather pour récupérer les temps correspondants
last_times = torch.gather(times, dim=1, index=last_indices)
print("Derniers temps pour type 1:\n", last_times)

Source tensor:
 tensor([[ 1,  2,  3,  4],
        [ 5,  6,  7,  8],
        [ 9, 10, 11, 12]])
Indices:
 tensor([[0, 2],
        [1, 3],
        [0, 3]])
Résultat gather:
 tensor([[ 1,  3],
        [ 6,  8],
        [ 9, 12]])

Exemple avec événements:
Events:
 tensor([[0, 1, 0, 1, 0],
        [1, 0, 0, 1, 1],
        [1, 1, 1, 0, 1]])
Times:
 tensor([[0.0917, 1.1058, 2.4856, 3.1408, 4.7153],
        [0.3074, 0.8413, 2.3905, 2.6149, 3.0617],
        [1.5503, 1.8348, 3.1659, 4.4744, 6.1252]])
Derniers indices pour type 1:
 tensor([[3],
        [4],
        [4]])
Derniers temps pour type 1:
 tensor([[3.1408],
        [3.0617],
        [6.1252]])


## 3. Indexation avec des masques booléens

In [11]:
# Masques booléens pour filtrage et sélection conditionnelle
values = torch.randn(4, 6)
print("Valeurs:\n", values)

# Masque pour valeurs positives
positive_mask = values > 0
print("\nMasque positif:\n", positive_mask)

# Sélectionner seulement les valeurs positives (retourne un tenseur 1D)
positive_values = values[positive_mask]
print("Valeurs positives:", positive_values[:10], "...")  # Affichage partiel

# Utilisation de masked_fill pour remplacer des valeurs
values_filled = values.masked_fill(values < 0, 0.0)
print("\nValeurs avec négatifs remplacés par 0:\n", values_filled)

# Utilisation de masked_select pour un filtrage plus complexe
mask_complex = (values > 0) & (values < 1)
selected = values.masked_select(mask_complex)
print("\nValeurs entre 0 et 1:", selected)

# Exemple pratique : masquer les paddings dans une séquence
seq_len = torch.tensor([3, 5, 2, 4])  # Longueurs réelles de chaque séquence
max_len = 5
batch_size = seq_len.size(0)

# Créer un masque de padding
padding_mask = torch.arange(max_len).unsqueeze(0) >= seq_len.unsqueeze(1)
print(f"\nMasque de padding (True = padding):\n{padding_mask}")

# Appliquer le masque sur des données
sequence_data = torch.randn(batch_size, max_len)
masked_data = sequence_data.masked_fill(padding_mask, float('-inf'))
print("Données avec padding masqué:\n", masked_data)

Valeurs:
 tensor([[ 1.4011,  1.5079, -1.0909,  0.4197,  0.3895,  0.5238],
        [-2.1130,  0.4523, -0.9354, -1.7169, -0.6213, -0.8477],
        [-0.8427,  0.7809,  0.7720, -1.2123,  1.2467,  0.7433],
        [-0.5526,  2.0318, -0.9890, -2.5797,  0.3550, -0.7316]])

Masque positif:
 tensor([[ True,  True, False,  True,  True,  True],
        [False,  True, False, False, False, False],
        [False,  True,  True, False,  True,  True],
        [False,  True, False, False,  True, False]])
Valeurs positives: tensor([1.4011, 1.5079, 0.4197, 0.3895, 0.5238, 0.4523, 0.7809, 0.7720, 1.2467,
        0.7433]) ...

Valeurs avec négatifs remplacés par 0:
 tensor([[1.4011, 1.5079, 0.0000, 0.4197, 0.3895, 0.5238],
        [0.0000, 0.4523, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.7809, 0.7720, 0.0000, 1.2467, 0.7433],
        [0.0000, 2.0318, 0.0000, 0.0000, 0.3550, 0.0000]])

Valeurs entre 0 et 1: tensor([0.4197, 0.3895, 0.5238, 0.4523, 0.7809, 0.7720, 0.7433, 0.3550])

Masque de paddi

## 4. Indexation avec torch.index_select et torch.take

In [12]:
# torch.index_select : sélectionner selon des indices sur une dimension
matrix = torch.randn(4, 6)
print("Matrice originale:\n", matrix)

# Sélectionner des lignes spécifiques
row_indices = torch.tensor([0, 2, 3])
selected_rows = torch.index_select(matrix, dim=0, index=row_indices)
print("\nLignes sélectionnées [0, 2, 3]:\n", selected_rows)

# Sélectionner des colonnes spécifiques
col_indices = torch.tensor([1, 3, 5])
selected_cols = torch.index_select(matrix, dim=1, index=col_indices)
print("\nColonnes sélectionnées [1, 3, 5]:\n", selected_cols)

# torch.take : traite le tenseur comme un tableau plat
flat_tensor = torch.arange(12).reshape(3, 4)
print("\nTenseur plat:\n", flat_tensor)

# Indices en tant que positions absolues
indices = torch.tensor([0, 5, 7, 11])
taken = torch.take(flat_tensor, indices)
print("Éléments pris aux positions [0, 5, 7, 11]:", taken)

# Exemple pratique : sélectionner des embeddings
vocab_size, embedding_dim = 1000, 128
embeddings = torch.randn(vocab_size, embedding_dim)

# Séquences d'indices de mots
word_indices = torch.tensor([[1, 5, 23, 7], [45, 2, 8, 12]])
batch_size, seq_length = word_indices.shape

# Sélectionner les embeddings correspondants
selected_embeddings = torch.index_select(embeddings, dim=0, 
                                        index=word_indices.flatten())
selected_embeddings = selected_embeddings.view(batch_size, seq_length, embedding_dim)

print(f"\nEmbeddings sélectionnés shape: {selected_embeddings.shape}")
print("Premier embedding du premier batch:", selected_embeddings[0, 0, :5])

Matrice originale:
 tensor([[ 1.4924, -1.5938, -0.9453,  1.2735,  0.4558,  0.8859],
        [-0.3927,  0.4843,  0.2566, -0.1102, -1.2561,  0.9141],
        [ 0.6275,  0.5636,  1.0326, -0.5624,  0.0315,  1.9188],
        [-0.3732, -0.0578,  0.3002,  2.2216, -0.0849, -0.6441]])

Lignes sélectionnées [0, 2, 3]:
 tensor([[ 1.4924, -1.5938, -0.9453,  1.2735,  0.4558,  0.8859],
        [ 0.6275,  0.5636,  1.0326, -0.5624,  0.0315,  1.9188],
        [-0.3732, -0.0578,  0.3002,  2.2216, -0.0849, -0.6441]])

Colonnes sélectionnées [1, 3, 5]:
 tensor([[-1.5938,  1.2735,  0.8859],
        [ 0.4843, -0.1102,  0.9141],
        [ 0.5636, -0.5624,  1.9188],
        [-0.0578,  2.2216, -0.6441]])

Tenseur plat:
 tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]])
Éléments pris aux positions [0, 5, 7, 11]: tensor([ 0,  5,  7, 11])

Embeddings sélectionnés shape: torch.Size([2, 4, 128])
Premier embedding du premier batch: tensor([ 1.5345, -0.7401,  0.4479,  0.8496,  0.0639])


## 5. Indexation multi-dimensionnelle avancée

In [13]:
# Indexation avec plusieurs dimensions simultanément
batch_size, num_classes, seq_len = 2, 3, 4
tensor_3d = torch.randn(batch_size, num_classes, seq_len)
print("Tenseur 3D shape:", tensor_3d.shape)
print("Données:\n", tensor_3d)

# Indexation fancy avec des listes de tenseurs
batch_idx = torch.tensor([0, 1, 0, 1])  # indices de batch
class_idx = torch.tensor([1, 2, 0, 1])  # indices de classe
seq_idx = torch.tensor([0, 1, 2, 3])    # indices de séquence

# Sélection d'éléments spécifiques
selected_elements = tensor_3d[batch_idx, class_idx, seq_idx]
print("\nÉléments sélectionnés:", selected_elements)

# Utilisation d'arange pour l'indexation
batch_range = torch.arange(batch_size)
print(f"Range batch: {batch_range}")

# Sélectionner le maximum de chaque batch dans chaque classe
max_values, max_indices = tensor_3d.max(dim=2)  # max sur seq_len
print(f"\nMax values shape: {max_values.shape}")
print("Max values:\n", max_values)
print("Max indices:\n", max_indices)

# Utiliser les indices pour récupérer les valeurs originales
batch_expanded = batch_range.unsqueeze(1).expand(-1, num_classes)
class_expanded = torch.arange(num_classes).unsqueeze(0).expand(batch_size, -1)

recovered_values = tensor_3d[batch_expanded, class_expanded, max_indices]
print("\nValeurs récupérées (vérification):\n", recovered_values)
print("Égalité:", torch.allclose(max_values, recovered_values))

Tenseur 3D shape: torch.Size([2, 3, 4])
Données:
 tensor([[[ 1.2225,  0.0716,  0.3245, -0.3487],
         [-1.2330,  0.8190,  0.9548,  1.0756],
         [ 0.3341,  0.4133, -0.0394,  0.3766]],

        [[ 0.6936, -0.5983, -0.5106,  0.8924],
         [ 0.6499,  0.7171, -0.9614,  0.5521],
         [-0.2948,  0.6610, -0.2536, -0.1455]]])

Éléments sélectionnés: tensor([-1.2330,  0.6610,  0.3245,  0.5521])
Range batch: tensor([0, 1])

Max values shape: torch.Size([2, 3])
Max values:
 tensor([[1.2225, 1.0756, 0.4133],
        [0.8924, 0.7171, 0.6610]])
Max indices:
 tensor([[0, 3, 1],
        [3, 1, 1]])

Valeurs récupérées (vérification):
 tensor([[1.2225, 1.0756, 0.4133],
        [0.8924, 0.7171, 0.6610]])
Égalité: True


## 6. Exemples appliqués aux processus ponctuels

In [14]:
# Applications pratiques pour les processus ponctuels
batch_size, max_seq_len, num_event_types = 3, 8, 4

# Simulation d'événements avec longueurs variables
seq_lengths = torch.tensor([5, 7, 6])
event_types = torch.randint(0, num_event_types, (batch_size, max_seq_len))
event_times = torch.randn(batch_size, max_seq_len).abs().cumsum(dim=1)

print("Types d'événements:\n", event_types)
print("Temps d'événements:\n", event_times)

# 1. Masquer les événements de padding
padding_mask = torch.arange(max_seq_len).unsqueeze(0) < seq_lengths.unsqueeze(1)
valid_events = event_types * padding_mask.long()  # 0 pour les paddings
valid_times = event_times.masked_fill(~padding_mask, 0.0)

print("\nÉvénements valides (avec masque):\n", valid_events)
print("Temps valides:\n", valid_times)

# 2. Trouver le dernier événement de chaque type pour chaque séquence
last_event_times = torch.zeros(batch_size, num_event_types)

for batch_idx in range(batch_size):
    for event_type in range(num_event_types):
        # Masque pour ce type d'événement dans cette séquence
        type_mask = (event_types[batch_idx] == event_type) & padding_mask[batch_idx]
        
        if type_mask.any():
            # Trouver l'index du dernier événement de ce type
            last_idx = torch.where(type_mask)[0][-1]
            last_event_times[batch_idx, event_type] = event_times[batch_idx, last_idx]

print("\nDerniers temps par type d'événement:\n", last_event_times)

# 3. Version vectorisée plus efficace
def get_last_event_times_vectorized(event_types, event_times, padding_mask, num_event_types):
    batch_size, seq_len = event_types.shape
    last_times = torch.zeros(batch_size, num_event_types)
    
    for event_type in range(num_event_types):
        # Masque pour ce type d'événement
        type_mask = (event_types == event_type) & padding_mask
        
        # Créer un tenseur avec -inf où il n'y a pas d'événements de ce type
        masked_times = event_times.masked_fill(~type_mask, float('-inf'))
        
        # Prendre le maximum (dernier temps) par batch
        max_times, _ = masked_times.max(dim=1)
        
        # Remplacer -inf par 0 si aucun événement de ce type
        valid_mask = max_times != float('-inf')
        last_times[valid_mask, event_type] = max_times[valid_mask]
    
    return last_times

last_times_vec = get_last_event_times_vectorized(event_types, event_times, padding_mask, num_event_types)
print("\nVersion vectorisée:\n", last_times_vec)
print("Égalité avec version naive:", torch.allclose(last_event_times, last_times_vec))

Types d'événements:
 tensor([[1, 3, 1, 2, 2, 3, 3, 3],
        [3, 1, 3, 1, 2, 0, 3, 0],
        [2, 1, 3, 2, 1, 2, 0, 1]])
Temps d'événements:
 tensor([[1.0120, 1.7871, 3.0567, 4.7263, 5.7100, 6.1736, 6.5166, 6.6035],
        [0.2779, 0.9679, 4.2464, 4.7372, 4.9811, 7.8192, 8.9795, 9.7267],
        [0.2971, 1.2268, 2.4551, 2.5715, 4.0258, 4.2429, 5.2187, 5.3796]])

Événements valides (avec masque):
 tensor([[1, 3, 1, 2, 2, 0, 0, 0],
        [3, 1, 3, 1, 2, 0, 3, 0],
        [2, 1, 3, 2, 1, 2, 0, 0]])
Temps valides:
 tensor([[1.0120, 1.7871, 3.0567, 4.7263, 5.7100, 0.0000, 0.0000, 0.0000],
        [0.2779, 0.9679, 4.2464, 4.7372, 4.9811, 7.8192, 8.9795, 0.0000],
        [0.2971, 1.2268, 2.4551, 2.5715, 4.0258, 4.2429, 0.0000, 0.0000]])

Derniers temps par type d'événement:
 tensor([[0.0000, 3.0567, 5.7100, 1.7871],
        [7.8192, 4.7372, 4.9811, 8.9795],
        [0.0000, 4.0258, 4.2429, 2.4551]])

Version vectorisée:
 tensor([[0.0000, 3.0567, 5.7100, 1.7871],
        [7.8192, 4.7372,

## 7. Techniques d'indexation pour l'optimisation

In [15]:
# Techniques avancées pour optimiser les performances

# 1. Éviter les boucles Python avec broadcasting
def naive_distance_matrix(points):
    """Version naive avec boucles (LENT)"""
    n = points.shape[0]
    distances = torch.zeros(n, n)
    for i in range(n):
        for j in range(n):
            distances[i, j] = torch.norm(points[i] - points[j])
    return distances

def vectorized_distance_matrix(points):
    """Version vectorisée (RAPIDE)"""
    # points shape: [n, d]
    # Utilisation du broadcasting
    diff = points.unsqueeze(1) - points.unsqueeze(0)  # [n, n, d]
    distances = torch.norm(diff, dim=2)  # [n, n]
    return distances

# Test avec des points aléatoires
points = torch.randn(100, 3)

# Mesure du temps
import time

start = time.time()
dist_vectorized = vectorized_distance_matrix(points)
time_vectorized = time.time() - start

print(f"Temps vectorisé: {time_vectorized:.4f}s")
print(f"Shape résultat: {dist_vectorized.shape}")

# 2. Indexation efficace avec scatter operations
batch_size, num_bins = 4, 10
values = torch.randn(batch_size, 20)  # 20 valeurs par batch
bin_indices = torch.randint(0, num_bins, (batch_size, 20))

print("\nValeurs à grouper par bins:", values[0, :5])
print("Indices de bins:", bin_indices[0, :5])

# Sommer les valeurs par bin avec scatter_add
bin_sums = torch.zeros(batch_size, num_bins)
bin_sums.scatter_add_(1, bin_indices, values)

print("Sommes par bin:\n", bin_sums)

# 3. Optimisation mémoire avec views et squeeze/unsqueeze
large_tensor = torch.randn(1000, 1000)
print(f"Tenseur original shape: {large_tensor.shape}")

# View pour reshaper sans copier
reshaped = large_tensor.view(100, 10, 100, 10)
print(f"Reshaped shape: {reshaped.shape}")
print(f"Même mémoire: {reshaped.data_ptr() == large_tensor.data_ptr()}")

# Permute + contiguous pour réorganiser efficacement
permuted = reshaped.permute(0, 2, 1, 3)  # Change l'ordre des dimensions
flattened = permuted.contiguous().view(100, 100, 100)  # Nécessite contiguous()
print(f"Final shape: {flattened.shape}")

# 4. Indexation avec topk pour sélection efficace
scores = torch.randn(5, 1000)  # 5 samples, 1000 features each
k = 10

# Sélectionner les top-k scores pour chaque sample
top_values, top_indices = torch.topk(scores, k, dim=1)
print(f"\nTop-{k} values shape: {top_values.shape}")
print(f"Top-{k} indices shape: {top_indices.shape}")

# Utiliser les indices pour récupérer d'autres informations
features = torch.randn(5, 1000, 64)  # Features associées
top_features = torch.gather(features, 1, 
                           top_indices.unsqueeze(-1).expand(-1, -1, 64))
print(f"Top features shape: {top_features.shape}")

print("\n=== Conseils d'optimisation ===")
print("1. Utilisez view() au lieu de reshape() quand possible")
print("2. Évitez les boucles Python, préférez les opérations vectorisées")
print("3. Utilisez scatter/gather pour les opérations groupées")
print("4. masked_fill est plus rapide que l'indexation conditionnelle")
print("5. topk est optimisé pour la sélection des meilleurs éléments")

Temps vectorisé: 0.0014s
Shape résultat: torch.Size([100, 100])

Valeurs à grouper par bins: tensor([ 0.9677,  0.8307, -0.6004, -0.0969, -0.9670])
Indices de bins: tensor([2, 3, 2, 9, 3])
Sommes par bin:
 tensor([[ 2.1389e+00,  0.0000e+00,  4.1902e+00,  1.9977e-03,  1.0681e+00,
          2.3102e-01, -7.2568e-01,  5.6631e-01,  0.0000e+00, -1.8103e-01],
        [ 0.0000e+00,  2.7782e+00, -1.4263e+00,  0.0000e+00,  6.3193e-01,
          3.1381e+00, -1.1786e-01,  4.7861e-01,  4.2729e-01,  2.0775e+00],
        [-2.1165e+00,  1.6351e+00,  4.4144e-01, -5.4236e-01,  5.4693e-01,
          0.0000e+00,  4.1478e-01, -1.1112e-01,  3.2264e+00, -3.7155e-01],
        [ 1.4515e+00, -1.0545e+00,  8.9650e-01,  2.7579e+00, -1.7292e-01,
         -8.0008e-01,  1.1697e-01, -1.7198e+00, -1.1725e+00, -1.5216e-01]])
Tenseur original shape: torch.Size([1000, 1000])
Reshaped shape: torch.Size([100, 10, 100, 10])
Même mémoire: True
Final shape: torch.Size([100, 100, 100])

Top-10 values shape: torch.Size([5, 10])


## 8. Différences entre gather, index_select et take

In [16]:
# Comparaison détaillée entre gather, index_select et take
import torch

# Créons un tenseur de référence pour tous les exemples
data = torch.tensor([
    [10, 11, 12, 13, 14],
    [20, 21, 22, 23, 24],
    [30, 31, 32, 33, 34],
    [40, 41, 42, 43, 44]
])
print("Tenseur de référence (4x5):")
print(data)
print("Shape:", data.shape)

print("\n" + "="*60)
print("1. TORCH.GATHER - Indexation flexible par dimension")
print("="*60)

# gather: sélectionne des éléments selon des indices, en préservant la structure
indices_gather = torch.tensor([
    [1, 3, 0, 4],  # Pour la ligne 0: colonnes 1,3,0,4
    [4, 2, 1, 0],  # Pour la ligne 1: colonnes 4,2,1,0  
    [2, 2, 2, 2],  # Pour la ligne 2: colonne 2 répétée
    [0, 1, 2, 3]   # Pour la ligne 3: colonnes 0,1,2,3
])

print("Indices pour gather:")
print(indices_gather)

gathered = torch.gather(data, dim=1, index=indices_gather)
print("Résultat gather (dim=1):")
print(gathered)
print("Shape:", gathered.shape)

print("\nCaractéristiques de gather:")
print("- Préserve le nombre de dimensions")
print("- Les indices peuvent être différents pour chaque 'ligne' (batch)")
print("- Shape du résultat = shape des indices")
print("- Très flexible pour l'indexation par batch")

Tenseur de référence (4x5):
tensor([[10, 11, 12, 13, 14],
        [20, 21, 22, 23, 24],
        [30, 31, 32, 33, 34],
        [40, 41, 42, 43, 44]])
Shape: torch.Size([4, 5])

1. TORCH.GATHER - Indexation flexible par dimension
Indices pour gather:
tensor([[1, 3, 0, 4],
        [4, 2, 1, 0],
        [2, 2, 2, 2],
        [0, 1, 2, 3]])
Résultat gather (dim=1):
tensor([[11, 13, 10, 14],
        [24, 22, 21, 20],
        [32, 32, 32, 32],
        [40, 41, 42, 43]])
Shape: torch.Size([4, 4])

Caractéristiques de gather:
- Préserve le nombre de dimensions
- Les indices peuvent être différents pour chaque 'ligne' (batch)
- Shape du résultat = shape des indices
- Très flexible pour l'indexation par batch


In [17]:
print("\n" + "="*60)
print("2. TORCH.INDEX_SELECT - Sélection de tranches entières")
print("="*60)

# index_select: sélectionne des lignes/colonnes ENTIÈRES selon des indices
indices_rows = torch.tensor([0, 2, 3])  # Sélectionner lignes 0, 2, 3
selected_rows = torch.index_select(data, dim=0, index=indices_rows)
print("Sélection de lignes [0, 2, 3]:")
print(selected_rows)
print("Shape:", selected_rows.shape)

indices_cols = torch.tensor([1, 4, 2])  # Sélectionner colonnes 1, 4, 2  
selected_cols = torch.index_select(data, dim=1, index=indices_cols)
print("\nSélection de colonnes [1, 4, 2]:")
print(selected_cols)
print("Shape:", selected_cols.shape)

print("\nCaractéristiques d'index_select:")
print("- Sélectionne des tranches COMPLÈTES selon une dimension")
print("- Les indices s'appliquent à TOUTES les lignes/colonnes uniformément")
print("- Plus simple que gather mais moins flexible")
print("- Équivalent au slicing avancé: data[[0,2,3], :] ou data[:, [1,4,2]]")


2. TORCH.INDEX_SELECT - Sélection de tranches entières
Sélection de lignes [0, 2, 3]:
tensor([[10, 11, 12, 13, 14],
        [30, 31, 32, 33, 34],
        [40, 41, 42, 43, 44]])
Shape: torch.Size([3, 5])

Sélection de colonnes [1, 4, 2]:
tensor([[11, 14, 12],
        [21, 24, 22],
        [31, 34, 32],
        [41, 44, 42]])
Shape: torch.Size([4, 3])

Caractéristiques d'index_select:
- Sélectionne des tranches COMPLÈTES selon une dimension
- Les indices s'appliquent à TOUTES les lignes/colonnes uniformément
- Plus simple que gather mais moins flexible
- Équivalent au slicing avancé: data[[0,2,3], :] ou data[:, [1,4,2]]


In [18]:
print("\n" + "="*60)
print("3. TORCH.TAKE - Indexation en tableau plat")
print("="*60)

# take: traite le tenseur comme un vecteur plat (1D)
print("Tenseur aplati conceptuellement:")
flat_view = data.flatten()
print("Position: ", list(range(len(flat_view))))
print("Valeur:   ", flat_view.tolist())

indices_take = torch.tensor([0, 5, 7, 12, 19])  # Positions absolues dans le tableau plat
taken = torch.take(data, indices_take)
print(f"\nIndices take: {indices_take.tolist()}")
print("Résultat take:", taken)
print("Shape:", taken.shape)

print("\nVérification manuelle:")
for i, idx in enumerate(indices_take):
    row = idx // 5  # Division entière pour trouver la ligne
    col = idx % 5   # Modulo pour trouver la colonne
    print(f"Index {idx} -> position [{row}, {col}] -> valeur {data[row, col]}")

print("\nCaractéristiques de take:")
print("- Traite le tenseur comme un tableau 1D (ordre row-major)")
print("- Indices en positions absolutes")
print("- Résultat toujours 1D")
print("- Utile pour indexation sparse ou échantillonnage aléatoire")


3. TORCH.TAKE - Indexation en tableau plat
Tenseur aplati conceptuellement:
Position:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
Valeur:    [10, 11, 12, 13, 14, 20, 21, 22, 23, 24, 30, 31, 32, 33, 34, 40, 41, 42, 43, 44]

Indices take: [0, 5, 7, 12, 19]
Résultat take: tensor([10, 20, 22, 32, 44])
Shape: torch.Size([5])

Vérification manuelle:
Index 0 -> position [0, 0] -> valeur 10
Index 5 -> position [1, 0] -> valeur 20
Index 7 -> position [1, 2] -> valeur 22
Index 12 -> position [2, 2] -> valeur 32
Index 19 -> position [3, 4] -> valeur 44

Caractéristiques de take:
- Traite le tenseur comme un tableau 1D (ordre row-major)
- Indices en positions absolutes
- Résultat toujours 1D
- Utile pour indexation sparse ou échantillonnage aléatoire


In [19]:
print("\n" + "="*60)
print("4. COMPARAISON PRATIQUE - Même objectif, approches différentes")
print("="*60)

print("OBJECTIF: Sélectionner la 2e et 4e colonne de toutes les lignes\n")

# Méthode 1: avec index_select (LE PLUS SIMPLE)
cols_wanted = torch.tensor([1, 3])
result_index_select = torch.index_select(data, dim=1, index=cols_wanted)
print("1. Avec index_select:")
print(result_index_select)

# Méthode 2: avec gather (PLUS VERBEUX mais plus flexible)
# Il faut répéter les indices pour chaque ligne
gather_indices = cols_wanted.unsqueeze(0).expand(data.shape[0], -1)
print(f"\nIndices pour gather (répétés): \n{gather_indices}")
result_gather = torch.gather(data, dim=1, index=gather_indices)
print("2. Avec gather (équivalent):")
print(result_gather)

# Méthode 3: avec take (COMPLIQUÉ)
# Il faut calculer les positions absolutes
take_indices = []
for row in range(data.shape[0]):
    for col in [1, 3]:  # colonnes 1 et 3
        take_indices.append(row * data.shape[1] + col)
take_indices = torch.tensor(take_indices)
result_take = torch.take(data, take_indices).reshape(data.shape[0], 2)
print("3. Avec take (plus complexe):")
print(result_take)

print(f"\nVérification - tous égaux: {torch.equal(result_index_select, result_gather) and torch.equal(result_gather, result_take)}")


4. COMPARAISON PRATIQUE - Même objectif, approches différentes
OBJECTIF: Sélectionner la 2e et 4e colonne de toutes les lignes

1. Avec index_select:
tensor([[11, 13],
        [21, 23],
        [31, 33],
        [41, 43]])

Indices pour gather (répétés): 
tensor([[1, 3],
        [1, 3],
        [1, 3],
        [1, 3]])
2. Avec gather (équivalent):
tensor([[11, 13],
        [21, 23],
        [31, 33],
        [41, 43]])
3. Avec take (plus complexe):
tensor([[11, 13],
        [21, 23],
        [31, 33],
        [41, 43]])

Vérification - tous égaux: True


In [20]:
print("\n" + "="*60)
print("5. CAS D'USAGE TYPIQUES")
print("="*60)

print("📌 GATHER - Quand utiliser:")
print("✓ Indexation différente par batch (ex: derniers événements)")
print("✓ Sélection de éléments selon des critères dynamiques")
print("✓ Extraction de valeurs selon des indices calculés")
print("✓ Traitement par batch avec indices variables")

print("\n📌 INDEX_SELECT - Quand utiliser:")
print("✓ Sélection de lignes/colonnes complètes")
print("✓ Sous-échantillonnage uniforme")
print("✓ Réorganisation de dimensions")
print("✓ Quand tous les batches ont les mêmes indices")

print("\n📌 TAKE - Quand utiliser:")
print("✓ Échantillonnage aléatoire de positions")
print("✓ Indexation sparse sur tenseurs aplatis")
print("✓ Conversion d'indices 2D vers 1D")
print("✓ Sélection d'éléments non-structurée")

# Exemple concret pour processus ponctuels
print("\n" + "="*60)
print("6. EXEMPLE PROCESSUS PONCTUELS")
print("="*60)

batch_size, seq_len, num_types = 3, 5, 2
events = torch.randint(0, num_types, (batch_size, seq_len))
times = torch.randn(batch_size, seq_len).abs().cumsum(dim=1)

print("Événements:")
print(events)
print("Temps:")
print(times)

# Cas 1: GATHER - Récupérer le temps du 3e événement de chaque batch
indices_3rd = torch.tensor([[2], [2], [2]])  # 3e position pour chaque batch
times_3rd_gather = torch.gather(times, dim=1, index=indices_3rd)
print(f"\n1. GATHER - 3e temps de chaque batch: {times_3rd_gather.flatten()}")

# Cas 2: INDEX_SELECT - Prendre les 2e et 4e temps de tous les batchs
positions = torch.tensor([1, 3])  # positions 2 et 4
times_selected = torch.index_select(times, dim=1, index=positions)
print("2. INDEX_SELECT - positions 2 et 4 pour tous:")
print(times_selected)

# Cas 3: TAKE - Récupérer des éléments específiques de façon sparse
sparse_indices = torch.tensor([1, 7, 12])  # positions absolues dans le tableau plat
times_sparse = torch.take(times, sparse_indices)
print(f"3. TAKE - éléments aux positions absolutes {sparse_indices.tolist()}: {times_sparse}")


5. CAS D'USAGE TYPIQUES
📌 GATHER - Quand utiliser:
✓ Indexation différente par batch (ex: derniers événements)
✓ Sélection de éléments selon des critères dynamiques
✓ Extraction de valeurs selon des indices calculés
✓ Traitement par batch avec indices variables

📌 INDEX_SELECT - Quand utiliser:
✓ Sélection de lignes/colonnes complètes
✓ Sous-échantillonnage uniforme
✓ Réorganisation de dimensions
✓ Quand tous les batches ont les mêmes indices

📌 TAKE - Quand utiliser:
✓ Échantillonnage aléatoire de positions
✓ Indexation sparse sur tenseurs aplatis
✓ Conversion d'indices 2D vers 1D
✓ Sélection d'éléments non-structurée

6. EXEMPLE PROCESSUS PONCTUELS
Événements:
tensor([[0, 1, 0, 0, 1],
        [1, 0, 1, 1, 0],
        [1, 0, 0, 1, 0]])
Temps:
tensor([[1.7946, 2.5892, 3.1408, 4.6366, 4.9139],
        [1.1127, 1.3986, 3.1231, 4.1274, 5.9714],
        [0.2518, 0.3949, 1.9774, 2.6786, 3.4462]])

1. GATHER - 3e temps de chaque batch: tensor([3.1408, 3.1231, 1.9774])
2. INDEX_SELECT - posi

In [21]:
print("\n" + "="*60)
print("7. TABLEAU RÉCAPITULATIF")
print("="*60)

print("""
┌─────────────────┬──────────────┬──────────────┬──────────────┐
│ Caractéristique │    GATHER    │ INDEX_SELECT │     TAKE     │
├─────────────────┼──────────────┼──────────────┼──────────────┤
│ Flexibilité     │    Haute     │   Moyenne    │    Faible    │
│ Complexité      │   Moyenne    │   Faible     │    Haute     │
│ Shape résultat  │ = shape idx  │ Préservée    │  Toujours 1D │
│ Indices/batch   │ Différents   │  Identiques  │   Absolus    │
│ Dimensions      │ Préservées   │ Dim-1 libre  │  Aplaties    │
│ Performance     │   Rapide     │   Rapide     │   Rapide     │
│ Cas d'usage     │ Batch varié  │ Slicing++    │ Sparse/1D    │
└─────────────────┴──────────────┴──────────────┴──────────────┘
""")

print("🎯 RÈGLE GÉNÉRALE:")
print("• Utilisez INDEX_SELECT pour des sélections uniformes simples")
print("• Utilisez GATHER pour des sélections par batch variables") 
print("• Utilisez TAKE pour des indexations 1D ou très spécifiques")

print("\n✨ CONSEIL PERFORMANCE:")
print("• Pour des sélections simples: slicing normal [indices] > index_select > gather")
print("• Pour des sélections complexes par batch: gather est optimal")
print("• Pour des accès sparse: take peut être utile mais souvent il y a mieux")


7. TABLEAU RÉCAPITULATIF

┌─────────────────┬──────────────┬──────────────┬──────────────┐
│ Caractéristique │    GATHER    │ INDEX_SELECT │     TAKE     │
├─────────────────┼──────────────┼──────────────┼──────────────┤
│ Flexibilité     │    Haute     │   Moyenne    │    Faible    │
│ Complexité      │   Moyenne    │   Faible     │    Haute     │
│ Shape résultat  │ = shape idx  │ Préservée    │  Toujours 1D │
│ Indices/batch   │ Différents   │  Identiques  │   Absolus    │
│ Dimensions      │ Préservées   │ Dim-1 libre  │  Aplaties    │
│ Performance     │   Rapide     │   Rapide     │   Rapide     │
│ Cas d'usage     │ Batch varié  │ Slicing++    │ Sparse/1D    │
└─────────────────┴──────────────┴──────────────┴──────────────┘

🎯 RÈGLE GÉNÉRALE:
• Utilisez INDEX_SELECT pour des sélections uniformes simples
• Utilisez GATHER pour des sélections par batch variables
• Utilisez TAKE pour des indexations 1D ou très spécifiques

✨ CONSEIL PERFORMANCE:
• Pour des sélections simples: slic